<a href="https://colab.research.google.com/github/mzucali/pyMinColab/blob/main/ExtractFromSummaryEMPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Marzo 2023 -
LEGGE IL FILE SUMMARY.TXT ed estrae DL e SD:

=>>input il file summary.txt o qualunque altro nome MA .TXT

In [1]:
# load and read summary file from EMPA
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving summary_short.txt to summary_short (2).txt
User uploaded file "summary_short (2).txt" with length 4114 bytes


In [2]:
num_elements = input("QUANTI ELEMENTI ANALIZZATI? ")
#num_elements = 11
loop = int(num_elements)+2

QUANTI ELEMENTI ANALIZZATI? 10


In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Mar 14 11:30:45 2023
GRAZIE ALESSANDRO COMUNIAN
@author: alex
"""

#file_in = open("summary.txt", "r")
file_in = open(fn,"r")
outFile = open("result2_tmp.txt", "w")

lines = file_in.readlines()

for i, line in enumerate(lines):
    # Just a spacing line
    #line_sp = line.split()
    #print(line_sp)
    line_s = line.split("\t")
    print(line_s)
    if line_s:
        try:
            if(line_s[2] == "Comment :"):
                print("line_s[2]",line_s[2])
                # Keep track of the comment value
                comm = line_s[3]
                # Just a spacing line
                print("comm",comm)
        except IndexError:
            pass

        if(line_s[0] == "Curr.(A) :"):
#        if(line_s[0] == "Element	"):
            print("ID {0} Comment".format(lines[i+1][:-1]))
            outFile.write("ID {0} Comment\n".format(lines[i+1][:-1]))
            for j in range(2,loop):
                print("{0} {1}\n".format(lines[i+j][:-1], comm), end="")
                outFile.write("{0} {1}\n".format(lines[i+j][:-1], comm))
                #outFile.write(" {0}\n".format(lines))

file_in.close()
outFile.close() #result2.txt



[' \n']
['Intensity & Mass%', '      Group : FabiolaCaso   ', '  Sample : 18-1-22         ', ' Page ', '  1 ', ' \n']
[' \n']
[' \n']
[' \n']
[' \n']
['Unknown Specimen', 'No.', '1', ' \n']
['Group :', 'FabiolaCaso', 'Sample :', '18-1-22', ' \n']
['UNK No. :', '1', 'Comment :', 'Line 1 AS584-S1-L1  \n']
line_s[2] Comment :
comm Line 1 AS584-S1-L1  

['Stage :', 'X=', '52.8390', 'Y=', '63.5565', 'Z=', '11.1440', ' \n']
['Acc. Voltage :', '15.0', '(kV)', 'Probe Dia. :', '0', 'Scan :', 'Off', ' \n']
['Dated on', 'Jan 18 10:35 2022', ' \n']
['WDS only', 'No. of accumulation :', '1', ' \n']
[' \n']
['Curr.(A) :', '5.003E-09', ' \n']
ID Element	Peak(mm)	Net(cps)	Bg-	Bg+	S.D.(%)	D.L.(ppm)	  Comment
1	Na	129.793	0.3	2.4	1.3	163.23		244	  Line 1 AS584-S1-L1  

2	Al	90.523	929.3	6.0	6.0	0.60	164	  Line 1 AS584-S1-L1  

3	Ti	87.989	0.3	6.9	7.2	267.36		321	  Line 1 AS584-S1-L1  

4	Mn	146.043	19.4	5.0	5.9	5.41	312	  Line 1 AS584-S1-L1  

5	K	119.659	-1.2	11.7	10.8	100.00		118	  Line 1 AS584-S1-L1 

In [4]:
#RESULT3 con spazi e tab corretti
newopen = open("result2_final.txt", "w")
with open("result2_tmp.txt") as f:
  first_line = f.readline()
  first_line_split = first_line.split()
  first_line = '\t'.join(first_line_split)+'\n'
  newopen.write(first_line)
  print(first_line)

with open("result2_tmp.txt") as f:
  for line in f:
    if 'ID Element	Peak(mm)' not in line:
      line_split = line.split()
      line = '\t'.join(line_split)+'\n'
      print(line)
      newopen.write(line)
newopen.close()

ID	Element	Peak(mm)	Net(cps)	Bg-	Bg+	S.D.(%)	D.L.(ppm)	Comment

1	Na	129.793	0.3	2.4	1.3	163.23	244	Line	1	AS584-S1-L1



2	Al	90.523	929.3	6.0	6.0	0.60	164	Line	1	AS584-S1-L1



3	Ti	87.989	0.3	6.9	7.2	267.36	321	Line	1	AS584-S1-L1



4	Mn	146.043	19.4	5.0	5.9	5.41	312	Line	1	AS584-S1-L1



5	K	119.659	-1.2	11.7	10.8	100.00	118	Line	1	AS584-S1-L1



6	Mg	107.805	215.5	4.7	3.2	1.27	189	Line	1	AS584-S1-L1



7	Si	77.271	1598.2	7.0	8.2	0.46	200	Line	1	AS584-S1-L1



8	Cr	73.260	0.3	13.6	9.3	370.25	339	Line	1	AS584-S1-L1



9	Fe	134.438	750.2	6.3	7.0	0.67	323	Line	1	AS584-S1-L1



10	Ca	107.373	188.1	15.3	14.6	1.46	134	Line	1	AS584-S1-L1



1	Na	129.793	-0.2	2.3	1.8	100.00	251	Line	2	AS584-S1-L1



2	Al	90.523	920.7	5.9	5.9	0.61	163	Line	2	AS584-S1-L1



3	Ti	87.989	0.9	5.8	5.8	85.11	290	Line	2	AS584-S1-L1



4	Mn	146.043	18.3	5.4	5.1	5.60	306	Line	2	AS584-S1-L1



5	K	119.659	-0.2	11.4	9.8	100.00	114	Line	2	AS584-S1-L1



6	Mg	107.805	221.7	6.1	3.2	1.25	201	Line	2	AS584-S1-L1



7	Si	77.

RESULT3.TXT con dati estratti di SD e DL

In [5]:
# EXTRACT X and Y coodinates
file_in2 = open(fn, "r")
lines = file_in2.readlines()
names = []
Xs = []
Ys = []
Zs = []

for i, line in enumerate(lines):
    line_s = line.split()
    #print(line_s)
    if line_s:
        try:
            if(line_s[4] == "Comment"):
                # Keep track of the comment value
                name = line_s[6]
                names.append(name)
                # Just a spacing line
            elif(line_s[0] == "Stage"):
                # Keep track of the comment value
                x = line_s[3]
                Xs.append(x)
                y = line_s[5]
                Ys.append(y)
                z = line_s[7]
                Zs.append(z)
                # Just a spacing line
        except IndexError:
            pass

import pandas as pd
# Just a spacing line
print("Sample: ",len(names))
print("X: ", len(Xs))
print("Y: ", len(Ys))
print("Z: ", len(Zs))
out = pd.DataFrame(
    {'Sample': names,
     'X':Xs,
     'Y':Ys,
     'Z':Zs
     })
# Just a spacing line
print(out)
out.to_csv('coordinate.csv', index=False)

Sample:  2
X:  2
Y:  2
Z:  2
  Sample        X        Y        Z
0   Line  52.8390  63.5565  11.1440
1   Line  52.8330  63.7800  11.1430


COORDINATE.CSV con sample name e coordinate X e Y

TUTTO DA **SCARICARE**